### 승하차인원과 인프라 데이터의 상관관계 살펴보기

In [ ]:
from utils import *
df = call_dataframe("")

# 인덱스는 버스 정류장

In [ ]:
# pairplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
%matplotlib inline

plt.rcParams['font.family'] = 'AppleGothic'

fig = sns.pairplot(df, hue=, palette='husl',markers=['o','s'],corner=True)

plt.show()

In [ ]:
# 히트맵 그리기
plt.figure(figsize=(15,8))
sns.heatmap(data=df.corr(),annot=True,
            fmt='.2f',linewidths=5,cmap='YIGnBu')
plt.title('',fontsize=15)
plt.show()

## PCA

In [ ]:
# 불필요한 경고 출력 방지
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# 시각화로 확인하기
# 꽃받침 시각화
sns.scatterplot(
    x=sepal_columns[0],
    y=sepal_columns[1],
    hue='target',
    data=df,
    palette="muted"
)

plt.title("Sepal")
plt.show()

In [ ]:
# For 반복문으로 고윳값과 기여율 확인하기
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

df = pd.DataFrame()

standard_scaler = StandardScaler()
standard_df = standard_scaler.fit_transform(df)

minmax_scaler = MinMaxScaler() 
minmax_df = minmax_scaler.fit_transform(df) 

robust_scaler = RobustScaler()
robust_df = robust_scaler.fit_transform(df)



### 적합한 n_components의 개수 찾기

In [ ]:
# 처음에는 DF의 피처 개수만큼 넣어서 기여율을 확인하다. 그 후의 적합한 component의 개수를 생각
pca = PCA(n_components = )
pca_array = pca.fit_transform(df)
pca_df = pd.DataFrame(pca_array,columns=[f'pca{num+1}' for num in range(df.shape[1])])
pca_df.head()

In [ ]:
# 주성분의 설명력과 기여율 구하기
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
                       '기여율':pca.explained_variance_ratio_},
                      index=np.array([f'pca{num+1}' for num in range(df.shape[1])]))
result['누적기여율'] = result['기여율'].cumsum()
result

In [ ]:
# scree plot

def scree_plot(pca):
    num_components = len(pca.explained_variance_ratio_)
    ind = np.arange(num_components)
    vals = pca.explained_variance_ratio_ 
    
    ax = plt.subplot()
    cumvals = np.cumsum(vals)
    ax.bar(ind, vals, color = ['#00da75', '#f1c40f',  '#ff6f15', '#3498db']) # Bar plot
    ax.plot(ind, cumvals, color = '#c0392b') # Line plot 
    
    for i in range(num_components): #라벨링(바 위에 텍스트(annotation) 쓰기)
        ax.annotate(r"%s" % ((str(vals[i]*100)[:3])), (ind[i], vals[i]), va = "bottom", ha = "center", fontsize = 13)
     
    ax.set_xlabel("PC")
    ax.set_ylabel("Variance")
    plt.title('Scree plot')
    
scree_plot(pca)

# 누적이 0.8이상인 부분까지

In [ ]:
# scree plot
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

### PCA 진행

In [ ]:
# 4차원 데이터를 2차원으로 축소
# n_components : 축소할 차원. 주성분 개수. 분산이 가장 큰 벡터의 개수
from pca import pca # biplot을 그리기 위한 별도 라이브러리

pca_2 = PCA(n_components=2)
pca_array = pca_2.fit_transform(df)
pca_2_df = pd.DataFrame(pca_array,
                      columns=['pca1','pca2'])
pca_2_df.head()

model = PCA(n_components=2)
results = model.fit_transform(df)
fig, ax = model.biplot(n_feat=2, legend =False)

In [ ]:
# 1번쨰 주성분을 오름차순으로 정렬해서 어떤 나라가 1번째 주성분 벡터값이 가장 높은지 살펴보기
# -는 벡터의 방향! 방향 상관없이 크기가 크면 된다
pca_1 = pca_2_df[['pca1']]
pca_1 = pca_1.sort_values(by='pca1', ascending=False)
pca_1 = pca_1.reset_index()
pca_1.index = pca_1.index+1
pca_1.index.name = "Ranking"
pca_1 = pca_1.rename(columns={'index':"station_name"})
pca_1

In [ ]:
# 2차원 시각화
sns.scatterplot(
    x="pac1",
    y="pca2",
    palette='muted',
    data= pca_2_df
)

plt.title("PCA n_component = 2")
plt.show()

In [ ]:
# 4차원에서 3차원으로 차원축소 해보기

pca_3 = PCA(n_components=3)

infra_pca_3 = pca_3.fit_transform(df.drop("target",axis=1))
infra_pca_3.shape

# PCA로 변환된 데이터를 DataFrame으로 만들기

pca_columns = ["Component 1", "Component 2", "Component 3"]
infra_pca_df = pd.DataFrame(
    infra_pca_3,
    columns=pca_columns
)

In [ ]:
# 3차원 시각화

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection='3d')

ax.scatter(infra_pca_df["Component 1"], # x축 데이터
           infra_pca_df["Component 2"], # y축 데이터
           infra_pca_df["Component 3"], # z축 데이터
        #    c=infra_pca_df['target'], # 3d scatter의 색깔 기준
           cmap=plt.cm.Set1, edgecolor='k', s=40)
ax.set_title("Iris 3D")
ax.set_xlabel("x")
ax.w_xaxis.set_ticklabels([]) # x축 tick 사용하지 않기
ax.set_ylabel("y")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("z")
ax.w_zaxis.set_ticklabels([])

plt.show()
# 4차원부터 시각화 불가능

In [ ]:
PC_values = np.arange(pca_3.n_components_) + 1
plt.plot(PC_values, pca_3.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
# 기존의 것과 pca를 한 것의 성능 차이 확인 할 것

### 클러스터링